In [5]:
import os
import pandas as pd

In [6]:
import tensorflow as tf 

In [7]:
os.chdir('../')
from models.dense import denseClass
from data import train_test_data, read_settings, normalize, tf_dataframe, WindowGenerator
from utils import save_json

In [8]:
settings = read_settings()
train, test = train_test_data(settings)

In [17]:
station_name = '(02000) cabecera autopista norte'
df = train[['dayofweek', 'weekday', station_name ]]
date = (2018,7,1)

In [28]:
w = WindowGenerator(input_width=10, label_width=3, 
                                    shift=3, data = df,label_columns=[station_name], 
                                    batch_size = 1, train_date=date)

In [29]:
for i, j in w.test:
    print (i)
    print(j)

tf.Tensor(
[[[3.0000e+00 1.0000e+00 8.2615e+04]
  [4.0000e+00 1.0000e+00 8.4939e+04]
  [5.0000e+00 0.0000e+00 6.3382e+04]
  [6.0000e+00 0.0000e+00 3.7378e+04]
  [0.0000e+00 1.0000e+00 8.0063e+04]
  [1.0000e+00 1.0000e+00 8.1507e+04]
  [2.0000e+00 1.0000e+00 8.3145e+04]
  [3.0000e+00 1.0000e+00 7.5230e+04]
  [4.0000e+00 1.0000e+00 8.2988e+04]
  [5.0000e+00 0.0000e+00 5.6799e+04]]], shape=(1, 10, 3), dtype=float32)
tf.Tensor(
[[[40239.]
  [37506.]
  [80489.]]], shape=(1, 3, 1), dtype=float32)
tf.Tensor(
[[[4.0000e+00 1.0000e+00 8.4939e+04]
  [5.0000e+00 0.0000e+00 6.3382e+04]
  [6.0000e+00 0.0000e+00 3.7378e+04]
  [0.0000e+00 1.0000e+00 8.0063e+04]
  [1.0000e+00 1.0000e+00 8.1507e+04]
  [2.0000e+00 1.0000e+00 8.3145e+04]
  [3.0000e+00 1.0000e+00 7.5230e+04]
  [4.0000e+00 1.0000e+00 8.2988e+04]
  [5.0000e+00 0.0000e+00 5.6799e+04]
  [6.0000e+00 0.0000e+00 4.0239e+04]]], shape=(1, 10, 3), dtype=float32)
tf.Tensor(
[[[37506.]
  [80489.]
  [84827.]]], shape=(1, 3, 1), dtype=float32)
tf.Tenso

In [4]:
data = pd.concat((train, test))
train_date = (2018,8,1)

train_index = data.index.get_loc(train_date)
input_widht = 14
OUT_STEPS = 7

In [ ]:
def run_model_online(model, train_station, 
                   test_station):

    data = pd.concat((train_station, test_station))
    train_index = data.index.get_loc(TRAIN_DATE)
    station_name = train.name


    predictions = []
    for i in data.index[train_index : train_index + 3]:
        new_train_index = data.index.get_loc(i)
        new_train_date = data.index[new_train_index]
        df = data[:new_train_index + OUT_STEPS]

        train_data = df[:new_train_index][station_name]
        train_mean = train_data.mean()
        train_std = train_data.std()

        df[station_name] = (df[station_name] - train_mean)/train_std

        w = WindowGenerator(input_width=input_widht, label_width=OUT_STEPS, 
                                    shift=OUT_STEPS, data = df,label_columns=[station_name], 
                                    batch_size = 32, train_date=new_train_date)

        if model == 'dense':
            model_class = denseClass(settings, station_name)
            model_class.fit(w)
            prediction = model_class.predict(w, train_mean, train_std)
            predictions.append(prediction)

        if model == 'cnn':
            model_class = cnnClass(settings, station_name)
            model_class.fit(w)
            prediction = model_class.predict(w, train_mean, train_std)

        if model == 'lstm':
            model_class = lstmClass(settings, station_name)
            model_class.fit(w)
            prediction = model_class.predict(w, train_mean, train_std)

    return predictions

In [5]:
predictions = []
for i in data.index[train_index : train_index + 3]:
    new_train_index = data.index.get_loc(i)
    new_train_date = data.index[new_train_index]
    
    df = data[:new_train_index + OUT_STEPS][['dayofweek','weekday','(02103) mazuren']]
    w = WindowGenerator(input_width=input_widht, label_width=OUT_STEPS, 
                                shift=OUT_STEPS, data = df,label_columns=['(02103) mazuren'], 
                                batch_size = 32, train_date=new_train_date)

    model_class = denseClass(settings, station_name = '(02103) mazuren')
    model_class.fit(w)
    prediction = model_class.predict(w, 0, 1)
    print (prediction)
    predictions.append(prediction)

[[[8540.153 ]
  [7743.7393]
  [7557.1997]
  [7539.3364]
  [7681.293 ]
  [8435.365 ]
  [9100.216 ]]]
[[[8641.156 ]
  [7820.134 ]
  [7626.103 ]
  [7608.2583]
  [7760.2817]
  [8538.563 ]
  [9219.463 ]]]
[[[8582.687 ]
  [7773.0376]
  [7581.6685]
  [7566.1987]
  [7714.59  ]
  [8479.897 ]
  [9151.342 ]]]


In [6]:
import numpy as np

In [16]:
np.array(predictions).reshape(-1, 7)

array([[8540.153 , 7743.7393, 7557.1997, 7539.3364, 7681.293 , 8435.365 ,
        9100.216 ],
       [8641.156 , 7820.134 , 7626.103 , 7608.2583, 7760.2817, 8538.563 ,
        9219.463 ],
       [8582.687 , 7773.0376, 7581.6685, 7566.1987, 7714.59  , 8479.897 ,
        9151.342 ]], dtype=float32)

In [15]:
predictions

[array([[[8540.153 ],
         [7743.7393],
         [7557.1997],
         [7539.3364],
         [7681.293 ],
         [8435.365 ],
         [9100.216 ]]], dtype=float32),
 array([[[8641.156 ],
         [7820.134 ],
         [7626.103 ],
         [7608.2583],
         [7760.2817],
         [8538.563 ],
         [9219.463 ]]], dtype=float32),
 array([[[8582.687 ],
         [7773.0376],
         [7581.6685],
         [7566.1987],
         [7714.59  ],
         [8479.897 ],
         [9151.342 ]]], dtype=float32)]

In [6]:
 w = WindowGenerator(input_width=input_widht, label_width=OUT_STEPS, 
                                shift=OUT_STEPS, data = df,label_columns=['(02103) mazuren'], 
                                batch_size = 32, train_date=train_date)

In [12]:
model_class = denseClass(settings, station_name = '(02103) mazuren')
model_class.fit(w)
prediction = model_class.predict(w, 0, 1)
prediction = prediction['prediction']

INFO:root:Model for station (02103) mazuren compleated. Time 0.05 minutes
INFO:root:Prediction for station (02103) mazuren compleated. Time 0.00 minutes


In [13]:
prediction


[[8471.33984375,
  7565.78369140625,
  7352.78564453125,
  7351.8427734375,
  7531.7646484375,
  8376.4677734375,
  9091.212890625]]